In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

import warnings
warnings.filterwarnings(action='once')

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('Transform_in_warehouse') \
    .config('spark.jars', 'spark-bigquery-with-dependencies_2.12-0.23.2.jar') \
    .getOrCreate()

22/03/23 14:41:47 WARN Utils: Your hostname, shura-tower resolves to a loopback address: 127.0.1.1; using 10.0.1.7 instead (on interface wlo1)
22/03/23 14:41:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/23 14:41:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
# Ran but didn't do anything
# spark.conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
# spark.conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

In [19]:
# Ran but gives different error
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

# Class com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem not found

https://cloud.google.com/dataproc/docs/tutorials/bigquery-connector-spark-example

### Example given

In [8]:
# # Use the Cloud Storage bucket for temporary BigQuery export data used
# # by the connector.
# bucket = "[bucket]"
# spark.conf.set('temporaryGcsBucket', bucket)


# # Load data from BigQuery.
# words = spark.read.format('bigquery') \
#   .option('table', 'bigquery-public-data:samples.shakespeare') \
#   .load()
# words.createOrReplaceTempView('words')

# # Perform word count.
# word_count = spark.sql(
#     'SELECT word, SUM(word_count) AS word_count FROM words GROUP BY word')
# word_count.show()
# word_count.printSchema()

# # Saving the data to BigQuery
# word_count.write.format('bigquery') \
#   .option('table', 'wordcount_dataset.wordcount_output') \
#   .save()

### My code

In [3]:
bucket = "dtc_data_lake_fresh-shell-338718"
spark.conf.set('temporaryGcsBucket', bucket)

In [4]:
feat_schema = types.StructType([
    types.StructField('song_id', types.StringType(), True),
    types.StructField('duration_ms', types.IntegerType(), True),
    types.StructField('key', types.IntegerType(), True),
    types.StructField('mode', types.IntegerType(), True),
    types.StructField('time_signature', types.IntegerType(), True),
    types.StructField('acousticness', types.DoubleType(), True),
    types.StructField('danceability', types.DoubleType(), True),
    types.StructField('energy', types.DoubleType(), True),
    types.StructField('instrumentalness', types.DoubleType(), True),
    types.StructField('liveness', types.DoubleType(), True),
    types.StructField('loudness', types.DoubleType(), True),
    types.StructField('speechiness', types.DoubleType(), True),
    types.StructField('valence', types.DoubleType(), True),
    types.StructField('tempo', types.DoubleType(), True)
])

In [5]:
# Song feature dataframe

df_feat = spark.read \
    .format('bigquery') \
    .schema(feat_schema) \
    .option('table', 'fresh-shell-338718.song_data.song_feature_data') \
    .load()

df_feat.createOrReplaceTempView('df_feat')

df_feat.printSchema()

22/03/23 14:42:10 WARN DefaultCredentialsProvider: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/.


root
 |-- song_id: string (nullable = true)
 |-- duration_ms: integer (nullable = true)
 |-- key: integer (nullable = true)
 |-- mode: integer (nullable = true)
 |-- time_signature: integer (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- danceability: double (nullable = true)
 |-- energy: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: double (nullable = true)
 |-- loudness: double (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- tempo: double (nullable = true)



In [6]:
pop_schema = types.StructType([
    types.StructField('song_id', types.StringType(), True),
    types.StructField('rank_score', types.IntegerType(), True),
    types.StructField('peak_position', types.IntegerType(), True),
    types.StructField('weeks_on_chart', types.IntegerType(), True),
    types.StructField('week', types.TimestampType(), True)
])

In [7]:
# Song popularity dataframe

df_pop = spark.read \
    .format('bigquery') \
    .schema(pop_schema) \
    .option('table', 'fresh-shell-338718.song_data.song_popularity_data') \
    .load()

df_pop.createOrReplaceTempView('df_pop')

df_pop.printSchema()

root
 |-- song_id: string (nullable = true)
 |-- rank_score: integer (nullable = true)
 |-- peak_position: integer (nullable = true)
 |-- weeks_on_chart: integer (nullable = true)
 |-- week: timestamp (nullable = true)



#### Transform tables

Feat: 20,405 rows - all distinct

Pop: 250,392 rows - only 20,402 songs

Inner joined: 250,392 rows, so all songs in pop are also in feat

In [12]:
spark.sql("""
SELECT
    count(*), tempo
FROM
    df_feat
GROUP BY
    tempo
SORT BY
    tempo
""").show(20, False)

+--------+------+
|count(1)|tempo |
+--------+------+
|1       |61.868|
|1       |62.254|
|1       |66.338|
|1       |66.757|
|1       |71.274|
|1       |75.272|
|1       |77.314|
|1       |79.956|
|1       |81.594|
|1       |82.199|
|1       |85.571|
|1       |86.576|
|1       |89.119|
|1       |90.029|
|1       |91.975|
|3       |91.999|
|3       |92.357|
|1       |95.998|
|1       |96.431|
|1       |97.245|
+--------+------+
only showing top 20 rows



In [9]:
spark.sql("""
SELECT
    *
FROM
    df_pop
WHERE
    song_id = '3GVhCuS4BRxCTvVj4Cmc8w'
""").show(20, False)

22/03/23 14:43:07 ERROR Executor: Exception in task 0.0 in stage 0.0 (TID 0)/ 1]
java.lang.UnsupportedOperationException
	at com.google.cloud.spark.bigquery.ArrowSchemaConverter$ArrowVectorAccessor.getLong(ArrowSchemaConverter.java:297)
	at com.google.cloud.spark.bigquery.ArrowSchemaConverter.getLong(ArrowSchemaConverter.java:98)
	at org.apache.spark.sql.vectorized.ColumnarBatchRow.getLong(ColumnarBatch.java:205)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:729)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$getByteArrayRdd$1(SparkPlan.scala:345)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:872)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2$adapted(RDD.scala:872)
	at org

Py4JJavaError: An error occurred while calling o64.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0, 10.0.1.7, executor driver): java.lang.UnsupportedOperationException
	at com.google.cloud.spark.bigquery.ArrowSchemaConverter$ArrowVectorAccessor.getLong(ArrowSchemaConverter.java:297)
	at com.google.cloud.spark.bigquery.ArrowSchemaConverter.getLong(ArrowSchemaConverter.java:98)
	at org.apache.spark.sql.vectorized.ColumnarBatchRow.getLong(ColumnarBatch.java:205)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:729)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$getByteArrayRdd$1(SparkPlan.scala:345

In [9]:
# Pared-down popularity dataset

df_pop_less = spark.sql("""
SELECT
    song_id, MAX(rank_score) as peak_position, MAX(weeks_on_chart) as weeks_on_chart
FROM
    df_pop
GROUP BY
    song_id
""")

df_pop_less.show(10, False)
# df_pop_less.printSchema()

+----------------------+-------------+--------------+
|song_id               |peak_position|weeks_on_chart|
+----------------------+-------------+--------------+
|09BND3I7P7WWC1Nax98P7G|30           |6             |
|1hbclSVBlZ2ipaoqOG9IOx|15           |4             |
|62ECcEUTVSnDvavP0al1ja|1            |2             |
|0JFc1TPOJtdrjytTMQa5Yh|96           |25            |
|5UY3Vzkc5INzJ3wBJm5FlD|2            |2             |
|74cWHGgHMuGGMg8KcEtR2X|2            |0             |
|2LWMF10STZqYY9a3YqBiOG|2            |0             |
|3sBuuFbaRBvFJWkd4Qgv7W|12           |8             |
|1DqIFLNEGRCGfJpkJzFX0G|48           |10            |
|68JS5SFTnW5Yv9Vzw81Jf0|6            |3             |
+----------------------+-------------+--------------+
only showing top 10 rows



In [14]:
# Joined dataset, feat and lesser pop

df_joined = spark.sql("""
SELECT
    f.song_id, duration_ms AS dur_ms, key, mode, time_signature AS time_sign, danceability, energy, loudness, valence, tempo,
    peak_pos, weeks
FROM
    df_feat AS f
INNER JOIN
    (SELECT song_id, MAX(rank_score) AS peak_pos, MAX(weeks_on_chart) AS weeks
    FROM df_pop
    GROUP BY song_id) AS pl
WHERE
    f.song_id = pl.song_id
""")

df_joined.show(10)
df_joined.printSchema()

+--------------------+------+---+----+---------+------------+------+--------+-------+-------+--------+-----+
|             song_id|dur_ms|key|mode|time_sign|danceability|energy|loudness|valence|  tempo|peak_pos|weeks|
+--------------------+------+---+----+---------+------------+------+--------+-------+-------+--------+-----+
|09BND3I7P7WWC1Nax...|161840|  0|   1|        4|       0.605| 0.576|  -8.943|  0.869|116.344|      30|    6|
|1hbclSVBlZ2ipaoqO...|135933|  7|   1|        4|       0.642| 0.404| -10.303|  0.646|112.071|      15|    4|
|62ECcEUTVSnDvavP0...|162200|  2|   1|        4|       0.514| 0.315| -13.969|  0.523|120.976|       1|    2|
|0JFc1TPOJtdrjytTM...|241973| 11|   0|        4|       0.557| 0.919|  -5.602|  0.804|108.524|      96|   25|
|5UY3Vzkc5INzJ3wBJ...|198840|  8|   1|        4|       0.538| 0.652|  -3.521|  0.515| 75.028|       2|    2|
|74cWHGgHMuGGMg8Kc...|192472|  4|   1|        4|       0.642| 0.606|  -4.678|  0.581|140.797|       2|    0|
|2LWMF10STZqYY9a3Y.

#### Save transformed tables to BQ

In [10]:
df_pop_less.write.format('bigquery') \
  .option('table', 'songs.chart_lesser') \
  .save()

Py4JJavaError: An error occurred while calling o64.save.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "gs"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3281)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3301)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:124)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3352)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3320)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:479)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:361)
	at com.google.cloud.spark.bigquery.BigQueryWriteHelper.<init>(BigQueryWriteHelper.scala:64)
	at com.google.cloud.spark.bigquery.BigQueryInsertableRelation.insert(BigQueryInsertableRelation.scala:42)
	at com.google.cloud.spark.bigquery.BigQueryRelationProvider.createRelation(BigQueryRelationProvider.scala:115)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSou

In [65]:
df_pop_less.write.format('bigquery') \
  .option('writeMethod', 'direct') \
  .save('songs.chart_lesser')

Py4JJavaError: An error occurred while calling o189.save.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "gs"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3281)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3301)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:124)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3352)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3320)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:479)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:361)
	at com.google.cloud.spark.bigquery.BigQueryWriteHelper.<init>(BigQueryWriteHelper.scala:64)
	at com.google.cloud.spark.bigquery.BigQueryInsertableRelation.insert(BigQueryInsertableRelation.scala:42)
	at com.google.cloud.spark.bigquery.BigQueryRelationProvider.createRelation(BigQueryRelationProvider.scala:115)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSo

In [ ]:
df_joined.write.format('bigquery') \
  .option('table', 'joined_feat_chart') \
  .save()